In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision as tv
from torchvision import datasets, models, transforms
from sklearn.metrics import roc_auc_score

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import time
import os
import copy
import requests
import io
import csv
import random
from timm.data import create_dataset, create_loader
from timm.scheduler import StepLRScheduler


plt.ion()   # interactive mode

import timm 
from tqdm import tqdm

In [2]:
# Check if CUDA (GPU support) is available
is_cuda_available = torch.cuda.is_available()
print("Is CUDA (GPU) available:", is_cuda_available)

# If CUDA is available, print the GPU name(s)
if is_cuda_available:
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPU(s) available: {gpu_count}")
    for i in range(gpu_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Using CPU.")

Is CUDA (GPU) available: True
Number of GPU(s) available: 1
GPU 0: Quadro RTX 6000


In [4]:
# config 
input_size = 3, 224, 224
img_size = 224
num_classes = 8
batch_size = 128

interpolation = 'bicubic'
DEFAULT_CROP_PCT = 1

test_dir = '../Dataset/images/test_caseonly_grouped'
class_map = {
    'Fluid_overload': 0,
    'Infection': 1,
    'Mass_Like_Lesions': 2,
    'Parenchymal_Disease': 3,
    'Atelectasis': 4,
    'Cardiomegaly': 5,
    'Pneumothorax': 6,
    'Pleural_Thickening': 7
    }

# create the train and etest datasets
test_dataset = create_dataset(name='', root=test_dir, split='validation', is_training=False, batch_size=batch_size, class_map = class_map)

test_len = len(test_dataset)
print('testidation set size: ' + str(test_len))

# resize images to fit the input of pretrained model
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop((224*3, 224*3)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
test_dataset.transform = transform

# create data loaders 
loader_test = create_loader(
        test_dataset,
        input_size=input_size,
        batch_size=batch_size,
        is_training=False,
        interpolation=interpolation,
        num_workers=8,
        pin_memory=True)

# check if labels are loaded as defined
test_dataset.reader.class_to_idx

testidation set size: 25726


{'Fluid_overload': 0,
 'Infection': 1,
 'Mass_Like_Lesions': 2,
 'Parenchymal_Disease': 3,
 'Atelectasis': 4,
 'Cardiomegaly': 5,
 'Pneumothorax': 6,
 'Pleural_Thickening': 7}

# Loading the ResNet50 Trained Model

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # if print 'cuda' then GPU is used

cuda


In [6]:
model_name = 'resnet50.a1_in1k'
num_classes = 8

model = timm.create_model(model_name, pretrained=False, num_classes=num_classes) #, img_size=img_size)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [10]:
ckpt_save_path = 'model_result/model_checkpoint/ResNet50-Multiclass-Caseonly-grouped/MODEL_CKPT_11_resnet50.a1_in1k_20240426_161059.pt'
checkpoint = torch.load(ckpt_save_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [11]:
model.eval()
torch.cuda.empty_cache()

# Predict

In [12]:
import torch.nn.functional as F

In [13]:
test_labels = np.empty((0,))
test_predictions = np.empty((0,))

with torch.no_grad():
    for images, labels in tqdm(loader_test):
        images = images.to(device)
        labels = labels.cpu().numpy()
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        predictions = predictions.cpu().numpy()

        test_labels = np.concatenate((test_labels, labels))
        test_predictions = np.concatenate((test_predictions, predictions))

100%|██████████| 201/201 [03:48<00:00,  1.13s/it]


In [14]:
# Get unique values and their counts
unique_values, counts = np.unique(test_labels, return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")


0.0: 5337
1.0: 7671
2.0: 3052
3.0: 1510
4.0: 3279
5.0: 1069
6.0: 2665
7.0: 1143


In [15]:
# Get unique values and their counts
unique_values, counts = np.unique(test_predictions, return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0.0: 3550
1.0: 12418
2.0: 4971
3.0: 719
4.0: 1886
5.0: 891
6.0: 1204
7.0: 87


In [16]:
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

In [17]:
# Accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy:", accuracy)

# Confusion matrix4
conf_matrix = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Classification report
class_report = classification_report(test_labels, test_predictions)
print("Classification Report:")
print(class_report)

Accuracy: 0.33689652491642696
Confusion Matrix:
[[1236 2689  638   78  378  157  139   22]
 [ 842 4722 1168  131  409  212  177   10]
 [ 274 1149 1284   70  109   43  112   11]
 [ 131  526  413  126  100   27  178    9]
 [ 526 1353  479   72  598  102  137   12]
 [ 110  498   82    5   58  310    4    2]
 [ 255 1026  603  191  181   19  380   10]
 [ 176  455  304   46   53   21   77   11]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.35      0.23      0.28      5337
         1.0       0.38      0.62      0.47      7671
         2.0       0.26      0.42      0.32      3052
         3.0       0.18      0.08      0.11      1510
         4.0       0.32      0.18      0.23      3279
         5.0       0.35      0.29      0.32      1069
         6.0       0.32      0.14      0.20      2665
         7.0       0.13      0.01      0.02      1143

    accuracy                           0.34     25726
   macro avg       0.28      0.25      0.2

In [18]:
# Convert true labels and predicted labels to binary format
test_labels_binary = label_binarize(test_labels, classes=[0, 1, 2, 3, 4, 5, 6, 7])
test_predictions_binary = label_binarize(test_predictions, classes=[0, 1, 2, 3, 4, 5, 6, 7])

# Compute OvR AUC
ovr_auc = roc_auc_score(test_labels_binary, test_predictions_binary, average='macro')
print("OvR AUC:", ovr_auc)

OvR AUC: 0.5705796880177005


In [19]:
# Compute OvR AUC for each class
class_auc_scores = []
for i in range(len(test_labels_binary[0])):
    class_test_labels = test_labels_binary[:, i]
    class_test_predictions = test_predictions_binary[:, i]
    class_auc = roc_auc_score(class_test_labels, class_test_predictions)
    class_auc_scores.append(class_auc)

# Print AUC for each class
for i, auc_score in enumerate(class_auc_scores):
    print(f"Class {i}: AUC = {auc_score:.4f}")

Class 0: AUC = 0.5590
Class 1: AUC = 0.5947
Class 2: AUC = 0.6290
Class 3: AUC = 0.5295
Class 4: AUC = 0.5625
Class 5: AUC = 0.6332
Class 6: AUC = 0.5534
Class 7: AUC = 0.5033
